# Training a single neuron

## Import Libraries

In [37]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

## Import Dataset

In [38]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [39]:
df.shape

(569, 33)

## Data Processing

In [40]:
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)

In [41]:
df.shape

(569, 31)

## Train Test Split

In [42]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2, random_state=42)

In [43]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(455, 30)
(455,)
(114, 30)
(114,)


## Scaling 

In [44]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [45]:
X_train

array([[-1.44075296, -0.43531947, -1.36208497, ...,  0.9320124 ,
         2.09724217,  1.88645014],
       [ 1.97409619,  1.73302577,  2.09167167, ...,  2.6989469 ,
         1.89116053,  2.49783848],
       [-1.39998202, -1.24962228, -1.34520926, ..., -0.97023893,
         0.59760192,  0.0578942 ],
       ...,
       [ 0.04880192, -0.55500086, -0.06512547, ..., -1.23903365,
        -0.70863864, -1.27145475],
       [-0.03896885,  0.10207345, -0.03137406, ...,  1.05001236,
         0.43432185,  1.21336207],
       [-0.54860557,  0.31327591, -0.60350155, ..., -0.61102866,
        -0.3345212 , -0.84628745]])

In [46]:
y_train[:10]

68     B
181    M
63     B
248    B
60     B
15     M
290    B
137    B
155    B
517    M
Name: diagnosis, dtype: object

## Label Encoding : Traget

In [47]:
encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [48]:
y_train[:10]

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 1])

In [49]:
X_train.shape

(455, 30)

## Numpy array to Pytorch Tensors

In [50]:
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [51]:
type(X_test)

torch.Tensor

## Define Model

In [68]:
class MySimpleNN(nn.Module):

    def __init__(self, num_features):

        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()
        
    # forward pass
    def forward(self, features):
        out = self.linear(features)
        out = self.sigmoid(out)
        return out
    

## Important Parameters

In [53]:
learning_rate = 0.1
epochs = 25

In [69]:
# define loss function
loss_function = nn.BCELoss()

In [55]:
# Ensure tensors are of the same dtype
X_train = X_train.float()  # Convert input to Float tensor
y_train = y_train.float()  # Convert labels to Float tensor

## Training Pipeline

In [73]:
# create model
model = MySimpleNN(X_train.shape[1])

# define optimizer:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loop
for epoch in range(epochs):
    
    # forward pass
    y_pred = model(X_train)

    # loss calculation
    loss = loss_function(y_pred, y_train.view(-1,1))

    # zeros gradients
    optimizer.zero_grad()
    
    # backward pass
    loss.backward()

    # parameter update
    optimizer.step()

    # print loss in each epoch
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.615679144859314
Epoch 2, Loss: 0.5060924887657166
Epoch 3, Loss: 0.43842053413391113
Epoch 4, Loss: 0.3921525478363037
Epoch 5, Loss: 0.35823214054107666
Epoch 6, Loss: 0.3320963680744171
Epoch 7, Loss: 0.3112028241157532
Epoch 8, Loss: 0.29402270913124084
Epoch 9, Loss: 0.27957993745803833
Epoch 10, Loss: 0.2672213613986969
Epoch 11, Loss: 0.25649216771125793
Epoch 12, Loss: 0.2470654547214508
Epoch 13, Loss: 0.23869949579238892
Epoch 14, Loss: 0.23121139407157898
Epoch 15, Loss: 0.2244599461555481
Epoch 16, Loss: 0.21833398938179016
Epoch 17, Loss: 0.21274477243423462
Epoch 18, Loss: 0.207620307803154
Epoch 19, Loss: 0.2029014229774475
Epoch 20, Loss: 0.1985391229391098
Epoch 21, Loss: 0.1944921463727951
Epoch 22, Loss: 0.190725639462471
Epoch 23, Loss: 0.18720978498458862
Epoch 24, Loss: 0.1839189976453781
Epoch 25, Loss: 0.18083105981349945


In [58]:
print("Weights shape", model.linear.weight.shape)
print("Bias shape", model.linear.bias.shape)

Weights shape torch.Size([1, 30])
Bias shape torch.Size([1])


In [61]:
print(model.linear.weight)

Parameter containing:
tensor([[-0.0399, -0.1087,  0.0896,  0.0716,  0.0067, -0.1160,  0.0913,  0.0922,
          0.0815,  0.0968,  0.1092,  0.1007, -0.2176,  0.0200, -0.0296, -0.1063,
         -0.1063,  0.0745,  0.0236,  0.0449, -0.0428,  0.0026,  0.0604, -0.0740,
          0.0917, -0.1142,  0.1231, -0.0523, -0.0147, -0.1036]],
       requires_grad=True)


## Evaluation

In [63]:
X_test = X_test.float()  # Convert input to Float tensor
y_test = y_test.float()  # Convert labels to Float tensor

In [64]:
with torch.no_grad():
    y_pred = model.forward(X_test)

y_pred

tensor([[0.4339],
        [0.3845],
        [0.4708],
        [0.4114],
        [0.4505],
        [0.3584],
        [0.4003],
        [0.4145],
        [0.4358],
        [0.3686],
        [0.4150],
        [0.4441],
        [0.4124],
        [0.5046],
        [0.4335],
        [0.4859],
        [0.4424],
        [0.4483],
        [0.5039],
        [0.4883],
        [0.3790],
        [0.4808],
        [0.4622],
        [0.4124],
        [0.3875],
        [0.5659],
        [0.3853],
        [0.4008],
        [0.4357],
        [0.2742],
        [0.4108],
        [0.4414],
        [0.4905],
        [0.4038],
        [0.4580],
        [0.4513],
        [0.3785],
        [0.4996],
        [0.4356],
        [0.3758],
        [0.4450],
        [0.4423],
        [0.4514],
        [0.3757],
        [0.5804],
        [0.4153],
        [0.5182],
        [0.3977],
        [0.4036],
        [0.3769],
        [0.4398],
        [0.4360],
        [0.3506],
        [0.6050],
        [0.3988],
        [0

In [65]:
# convert in 1 or 0
y_pred = (y_pred > 0.5).float()

In [66]:
y_pred

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
      

## Accuracy

In [67]:
accuracy = (y_pred == y_test).float().mean()

print(accuracy)

tensor(0.6034)
